In [1]:
import hashlib
import json

In [2]:
def hash_node(node):
    """Hashes a decision tree node (internal or leaf)."""

    # Convert node to JSON string for consistent hashing, handling different node types
    if isinstance(node, dict):  # Internal node
        node_str = json.dumps(node, sort_keys=True) # Sort keys for consistent hashing
    elif isinstance(node, (int, float, str, bool)): # Leaf node (value)
        node_str = str(node)
    else: # Handle other leaf node types if needed
        node_str = str(node)  # Or raise an exception if not supported

    return hashlib.sha256(node_str.encode()).hexdigest()

In [3]:
def build_merkle_tree(tree):
    """Builds a Merkle tree from a decision tree.

    Args:
        tree: A dictionary representing the decision tree.  The structure should be
            such that each node is a dictionary. For internal nodes, there should be
            keys like 'feature', 'threshold', 'left', 'right'. Leaf nodes can simply
            be the predicted value (e.g., a class label).

    Returns:
        The root hash of the Merkle tree, or None if the tree is empty.
    """

    if not tree:  # Handle empty tree
        return None

    def _build_merkle_tree_recursive(node):
        if isinstance(node, (int, float, str, bool)):  # Leaf node
            return hash_node(node)

        elif isinstance(node, dict): # Internal node
            left_hash = _build_merkle_tree_recursive(node.get('left'))
            right_hash = _build_merkle_tree_recursive(node.get('right'))

            if left_hash is None and right_hash is None: # Handle case of no children
                return hash_node(node) # Hash the node itself

            combined = (left_hash or "") + (right_hash or "") # Handle missing children
            return hash_node(combined)
        else:
            raise ValueError("Unsupported node type in tree")

    return _build_merkle_tree_recursive(tree)

In [4]:
empty_tree = {}
merkle_root_empty = build_merkle_tree(empty_tree)
print("Merkle Root (Empty):", merkle_root_empty)

Merkle Root (Empty): None


In [5]:
# Example with a single leaf node
leaf_node = 5
merkle_root_leaf = build_merkle_tree(leaf_node)  # Should raise error, not a dict
print("Merkle Root (Leaf):", merkle_root_leaf)

Merkle Root (Leaf): ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d


In [6]:
# Example with a simpler tree
simpler_tree = {
    'feature': 'X1',
    'threshold': 0.5,
    'left': 0,
    'right': 1
}

merkle_root_simple = build_merkle_tree(simpler_tree)
print("Merkle Root:", merkle_root_simple)

Merkle Root: fa13bb36c022a6943f37c638126a2c88fc8d008eb5a9fe8fcde17026807feae4


In [7]:
example_tree = {
    'feature': 'X1',
    'threshold': 0.5,
    'left': {
        'feature': 'X2',
        'threshold': 0.3,
        'left': 0,  # Leaf node (class 0)
        'right': 1   # Leaf node (class 1)
    },
    'right': {
        'feature': 'X3',
        'threshold': 0.7,
        'left': 1,  # Leaf node (class 1)
        'right': 0   # Leaf node (class 0)
    }
}

merkle_root = build_merkle_tree(example_tree)
print("Merkle Root:", merkle_root)

Merkle Root: 0de9ab1bd76d869a0b268033cf47e97fd46aa22e20e5d1f4e6d253714f8e0b05
